First and formost, create a series as example:

首先，创建一个多层目录series用于演示

In [1]:
import pandas as pd
import numpy as np

arrays = [['baz', 'baz', 'bar', 'bar', 'foo', 'foo', 'qux', 'qux'], ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
s = pd.Series(np.random.randn(8), index=index)
print(s)

first  second
baz    one       0.085324
       two      -0.008981
bar    one      -1.201993
       two       0.735646
foo    one       0.397933
       two      -1.269596
qux    one       0.461881
       two      -0.200283
dtype: float64


If we simply sort by python built-in functions, it will looks like that:

如果我们只是单纯地用python内置库进行排序，会变成这样：

In [2]:
s.sort_index(level = 'first').sort_values()

first  second
foo    two      -1.269596
bar    one      -1.201993
qux    two      -0.200283
baz    two      -0.008981
       one       0.085324
foo    one       0.397933
qux    one       0.461881
bar    two       0.735646
dtype: float64

It is not we want. What we need is to lock the index 'first' first, do sorting inside the index and than sort the whole bunch of data with different 'first'.

So convert series into dataframe first by using reset_index(). Dataframe provide a function also called sort_values(), because of the data type 'Dataframe' does not have multi-index, the different indexes in series, as well as values, will be considered as cells in dataframe

这并不是我们想要的结果（first那列乱了！！），我们需要先锁定first那列，然后对后面那堆数值进行排序，然后再将拥有相同first的项作为一个整体，对first进行排序（真正的排序结果在本文最后）。

所以要把series先转换成dataframe(用reset_index()这个函数转的)，这样就可以用reset_index方程根据排序优先级进行排序。因为dataframe没有多层目录这个东西，所有的目录名都被当成变量处理。我们把目录的优先级设置成最高就不会将目录打乱了

In [3]:
s_df = s.reset_index()
print(s_df)
print('\n Now the datatype of \'s_df\' is:')
print(type(s_df))

first second         0
0   baz    one  0.085324
1   baz    two -0.008981
2   bar    one -1.201993
3   bar    two  0.735646
4   foo    one  0.397933
5   foo    two -1.269596
6   qux    one  0.461881
7   qux    two -0.200283

 Now the datatype of 's_df' is:
<class 'pandas.core.frame.DataFrame'>


In [4]:
s_sorted_df = s_df.sort_values(by = ['first',0])
print(s_sorted_df)

first second         0
2   bar    one -1.201993
3   bar    two  0.735646
1   baz    two -0.008981
0   baz    one  0.085324
5   foo    two -1.269596
4   foo    one  0.397933
7   qux    two -0.200283
6   qux    one  0.461881


Now use this dataframe to rebuild a series, the new series is the outcome we want

现在就用这个排好序的Dataframe去重新建立Series，下面那堆实在是很麻烦，没有能直接转换的内置函数。

In [5]:
index1 = s_sorted_df.loc[:,'first'].tolist()
index2 = s_sorted_df.loc[:,'second'].tolist()
sorted_values = s_df.loc[:,0]

print('Data Type of the data under index1:')
print(type(index1))
print('\n')
print('Data Type of the data under index2:')
print(type(index2))
print('\n')
print('Data Type of the data under sorted_values:')
print(type(sorted_values))

Data Type of the data under index1:
<class 'list'>


Data Type of the data under index2:
<class 'list'>


Data Type of the data under sorted_values:
<class 'pandas.core.series.Series'>


values should be converted to numpy-array otherwise it cannot be used to from a series

需要注意的是，数值那列需要转换成numpy.array格式才能作为series的数值

In [6]:
sorted_values = np.array(sorted_values)
print('Now the datatype of values is:')
print(type(sorted_values))

Now the datatype of values is:
<class 'numpy.ndarray'>


Now assemble everything to form a series

然后把几个目录和数值弄到一起就好了。

In [7]:
sorted_index = [index1,index2]
sorted_s = pd.Series(sorted_values,index=sorted_index)
print(sorted_s)

bar  one    0.085324
     two   -0.008981
baz  two   -1.201993
     one    0.735646
foo  two    0.397933
     one   -1.269596
qux  two    0.461881
     one   -0.200283
dtype: float64


That is what we want.

完工